# Authentication Methods That Don't Work

**Purpose:** This notebook archives authentication methods tested for R ADBC connections that **do not work** in Snowflake Workspace Notebooks.

**Why archived:** These methods are blocked by SPCS security restrictions. They are preserved here in case Snowflake changes/adds support in the future.

## Summary

| Method | Status | Reason |
|--------|--------|--------|
| SPCS OAuth Token | ❌ Blocked | Token restricted to specific Snowflake connectors |
| Username/Password | ❌ Blocked | SPCS enforces OAuth for internal connections |
| External OAuth | ⚠️ Untested | Requires enterprise IdP integration |

## Working Methods

See the main `r_workspace_notebook.ipynb` for working authentication methods:
- **PAT (Programmatic Access Token)** - Recommended
- **Key Pair (JWT)** - Alternative

---

## Prerequisites

Before running these tests, ensure:
1. R environment is installed (`bash setup_r_environment.sh --adbc`)
2. `setup_r_environment()` has been run
3. Alternative auth functions are loaded

In [ ]:
# Load alternative auth test functions
from r_helpers import init_r_alt_auth

success, msg = init_r_alt_auth()
print(msg)

---

## OAuth Authentication Tests ❌ (SPCS Token Blocked)

This section tests OAuth-based authentication methods:

1. **SPCS OAuth Token** - The token injected into the container (`/snowflake/session/token`) **does NOT work** with ADBC. It's restricted to specific Snowflake connectors.

2. **External OAuth** - If your organization has an identity provider (Okta, Azure AD, etc.) configured with Snowflake, external OAuth tokens might work, but this is **untested**.

In [ ]:
from r_helpers import OAuthAuth

oauth = OAuthAuth()

# Option 1: Set external OAuth token (from your IdP)
# oauth.set_token("your-oauth-access-token-here")

# Option 2: Try SPCS token (expected to fail - for demonstration)
result = oauth.load_spcs_token()
print("SPCS Token Load Result:")
for key, value in result.items():
    print(f"  {key}: {value}")

In [ ]:
# Configure and test OAuth
if result['success']:
    config = oauth.configure_for_adbc()
    print("OAuth Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")

In [ ]:
%%R
# Test SPCS OAuth token (expected to FAIL)
# This demonstrates that the container's injected token cannot be used with ADBC
result <- test_spcs_token()
rprint(result)

### Expected Error

```
✗ OAuth authentication FAILED: IO: [Snowflake] 395092 (08004): Error connecting to Snowflake 
via Snowpark Container Services. Client is unauthorized to use Snowpark Container Services OAuth token.
```

---

## Username/Password Authentication ❌ (Blocked)

**Does NOT work** - SPCS enforces OAuth for connections from within containers.

This test is included for completeness to document the expected failure mode.

In [ ]:
%%R
# Test username/password authentication (expected to FAIL in SPCS)
# Uncomment and set password to test:
# Sys.setenv(SNOWFLAKE_PASSWORD = "your-password")
# result <- test_password_auth()
# rprint(result)

cat("Username/password test skipped (set SNOWFLAKE_PASSWORD to test)\n")
cat("Note: This is expected to fail with 'Please use OAuth when connecting to Snowflake'\n")

### Expected Error

```
✗ Password authentication FAILED (expected): IO: Please use OAuth when connecting to Snowflake
```

---

## Future Considerations

If Snowflake adds support for these methods in the future:

1. **SPCS OAuth Token**: Would require Snowflake to add ADBC to the list of authorized connectors for container OAuth tokens.

2. **Username/Password**: Would require SPCS to relax OAuth enforcement for outbound connections via the public endpoint.

3. **External OAuth**: Should theoretically work today if you have an IdP configured with Snowflake, but requires enterprise setup.